In [7]:
# This jupyter notebook is for UNSW COMP9517
# Assignment 01 Cell Detection

In [2]:
import cv2
import numpy as np 

In [13]:
# This function is to implement Otsu algorithm
# STEP 1 Compute histogram and probabilities of each intensity pixel
# STEP 2 Setup initial w0 and u0
# STEP 3 Step through all possible thresholds t=1,...maximum intensity
#           a. Update wi and ui b. Compute 𝜎
# STEP 4 Desired threshold corresponds to the maximum 𝜎


def Otsu(image_name):
    # Read the image from the dir
    image = cv2.imread(image_name,0)
    # STEP 1: calculating the histogram 
    frequency_vector = [0 for i in range(0,256)]
    frequency_vector = np.array(frequency_vector)
    # test the type of the list
    print(type(frequency_vector))
    row = image.shape[0]
    col = image.shape[1]
    for r in row:
        for c in col:
            frequency_vector[image[i,j]] += 1

256
